In [ ]:
import dolfinx
import ufl
import numpy as np
import matplotlib.pyplot as plt

from mpi4py import MPI
from petsc4py import PETSc

import time

In [ ]:
def main(params):
    comm = MPI.COMM_WORLD

    nx = 128

    # ================================================================
    # mesh, elements, function spaces, test and trial
    # ================================================================

    mesh = dolfinx.mesh.create_unit_square(comm, nx, nx)

    fe = ufl.FiniteElement("CG", mesh.ufl_cell(), degree=1)
    ve = ufl.VectorElement("CG", mesh.ufl_cell(), degree=2)

    Q = dolfinx.fem.FunctionSpace(mesh, fe)
    V = dolfinx.fem.FunctionSpace(mesh, ve)

    u = ufl.TrialFunction(Q)
    v = ufl.TestFunction(Q)

    temperature = dolfinx.fem.Function(Q, name="temperature")
    temperature_old = dolfinx.fem.Function(Q, name="temperature_old")
    temperature_older = dolfinx.fem.Function(Q, name="temperature_older")

    # ================================================================
    # Values that set the dynamics tradeoff/importance
    # ================================================================
    alpha = dolfinx.fem.Constant(mesh, params["alpha"])
    hot = params["hot_wall"]
    vel_scale = params["vel_scale"]

    # ================================================================
    # Make a dummy velocity field (Taylor-Green vortex)
    # ================================================================
    class VelocityAssigner:
        def __init__(self, vel_scale):
            self.vel_scale = vel_scale
            pass

        def __call__(self, x):
            vel = np.zeros((2, x.shape[1]), dtype=PETSc.ScalarType)

            # Taylor-Green Vortex in a box
            vel[0] = self.vel_scale * np.sin(x[0] * np.pi) * np.cos(x[1] * np.pi)
            vel[1] = self.vel_scale * -np.cos(x[0] * np.pi) * np.sin(x[1] * np.pi)

            return vel


    velocity = dolfinx.fem.Function(V, name="velocity")
    velocity.interpolate(VelocityAssigner(vel_scale))

    # ================================================================
    # Set boundary conditions on temperature walls
    # marker -> entities -> dofs -> BCs
    # ================================================================
    def _ylo_wall(x):
        return np.isclose(x[1], 0.0)

    def _yhi_wall(x):
        return np.isclose(x[1], 1.0)

    ylo_entities = dolfinx.mesh.locate_entities_boundary(mesh, dim=1, marker=_ylo_wall)
    yhi_entities = dolfinx.mesh.locate_entities_boundary(mesh, dim=1, marker=_yhi_wall)

    ylo_dofs = dolfinx.fem.locate_dofs_topological(Q, entity_dim=1, entities=ylo_entities)
    yhi_dofs = dolfinx.fem.locate_dofs_topological(Q, entity_dim=1, entities=yhi_entities)

    bcs = []

    bcs.append(dolfinx.fem.dirichletbc(hot, dofs=ylo_dofs, V=Q))
    bcs.append(dolfinx.fem.dirichletbc(-hot, dofs=yhi_dofs, V=Q))

    # ================================================================
    # Write the form: u \dot T - alpha * nabla ^ 2 T = 0
    # ================================================================
    constant_zero = dolfinx.fem.Constant(mesh, 0.0)
    
    dt = dolfinx.fem.Constant(mesh, 0.1)
    t_final = 50.0
    t_steps = int(t_final/dt.value)
    
    F = alpha * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    F += ufl.inner(ufl.dot(velocity, ufl.grad(u)), v) * ufl.dx
    F += (1/dt) * ufl.inner(u - temperature_old, v) * ufl.dx

    residual_form = 1
    
    # Residual, think this is just writing the "strong" governing equation?
    if residual_form == 1:
        # all terms
        r = (u - temperature_old)/dt + ufl.dot(velocity, ufl.nabla_grad(u)) - alpha*ufl.div(ufl.grad(u))
    elif residual_form == 2:
        # no dt term
        r = ufl.dot(velocity, ufl.nabla_grad(u)) - alpha*ufl.div(ufl.grad(u))
    elif residual_form == 3:
        # convection term only
        r = ufl.dot(velocity, ufl.nabla_grad(u))

    # Add SUPG stabilization, depends on the residual, velocity magnitude, and local mesh size
    vnorm = ufl.sqrt(ufl.dot(velocity, velocity))
    
    h = ufl.CellDiameter(mesh)
    
    delta = h/(2.0*vnorm)
    
    stab = delta * ufl.dot(velocity, ufl.grad(v)) * r * ufl.dx
    
    using_stability = True
    
    if using_stability:
        F += stab
    
    a = dolfinx.fem.form(ufl.lhs(F))
    L = dolfinx.fem.form(ufl.rhs(F))

    # ================================================================
    # Assemble and solve
    # ================================================================
    # Assemble the LHS matrix
    A = dolfinx.fem.petsc.assemble_matrix(a, bcs=bcs)
    A.assemble()

    # Assemble the RHS vector
    b = dolfinx.fem.petsc.assemble_vector(L)

    # Build the solver object for the temperature
    temperature_solver = PETSc.KSP().create(comm)

    temperature_solver.setOperators(A)

    temperature_solver.setType("preonly")
    temperature_solver.getPC().setType("lu")
    
#     temperature_solver.setType("cg")
#     temperature_solver.getPC().setType("jacobi")
#     temperature_solver.getPC().setHYPREType("euclid")
#     euclid, pilut, parasails, boomeramg, ams

    temperature_solver.view()
        
    temperature_solver.setFromOptions()
        
    filename = "low_alpha_temp_unsteady_withstab.xdmf"
    
    # Save the results
    with dolfinx.io.XDMFFile(comm, filename, "w") as fp:
        fp.write_mesh(mesh)
        fp.write_function(temperature, 0.0)
        fp.write_function(velocity, 0.0)
            
    for k in range(t_steps):
        
        A.zeroEntries()  # resets the matrix
        A = dolfinx.fem.petsc.assemble_matrix(A, a, bcs=bcs)
        A.assemble()
        temperature_solver.setOperators(A)

        with b.localForm() as loc:
            loc.set(0)
            b = dolfinx.fem.petsc.assemble_vector(b, L)

        # Apply boundary conditions
        dolfinx.fem.petsc.apply_lifting(b, [a], [bcs])

        b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)

        dolfinx.fem.petsc.set_bc(b, bcs)

        # Solve the system
        temperature_solver.solve(b, temperature.vector)
        temperature.x.scatter_forward()
        
        temperature_older.x.array[:] = temperature_old.x.array[:]
        temperature_old.x.array[:] = temperature.x.array[:]

        # ================================================================
        # Print outputs, save paraview files
        # ================================================================
        if (k+1) % 10 == 0:
            print(f"step {k+1} of {t_steps}")

            with dolfinx.io.XDMFFile(comm, filename, "a") as fp:
                fp.write_function(temperature, (k+1)*dt.value)
                fp.write_function(velocity, (k+1)*dt.value)
            
    for k, pt in enumerate(mesh.geometry.x):
        
        if np.isclose(pt[0], 0.5):
            plt.plot(pt[1], temperature.x.array[k], "x", color="C0")
            
    plt.title("Temperature profile along vertical midline")
    plt.xlabel("y-position")
    plt.ylabel("temperature")

    print(f"vel-to-alpha scaling: {params['vel_scale']/params['alpha']:.2f}")

    plt.show()


In [ ]:
params = {}
params["hot_wall"] = 100.0
params["vel_scale"] = 1.0
params["alpha"] = 1.0e-5

tic = time.time()
main(params)
toc = time.time()
print(toc-tic)